In [47]:

import pandas as pd
import seaborn as sb
import numpy as np
import math as m
import os
import csv
import random as rn
from matplotlib import pyplot as plt
import scipy as sp
from scipy.stats import truncnorm as tn

# Quantities to seed

papers_number = 20000
readers_number = 2500
authors_number = 25

papers = np.arange(papers_number)
readers = np.arange(readers_number)
authors = np.arange(authors_number)

# Seed folder path

dataset_name = "seed_2/p_4"
dataset_folder_path = f"../data/{dataset_name}/"
info_file_path = f"{dataset_folder_path}info.csv"
ratings_file_path = f"{dataset_folder_path}ratings.csv"
authors_file_path = f"{dataset_folder_path}authors.csv"

os.makedirs(dataset_folder_path, exist_ok=True)

print("DATASET NAME: ", dataset_name)
print("DATASET FOLDER PATH: ", dataset_folder_path)
print("INFO FILE PATH: ", info_file_path)
print("RATINGS FILE PATH: ", ratings_file_path)
print("AUTHORS FILE PATH: ", authors_file_path)


DATASET NAME:  seed_2/p_4
DATASET FOLDER PATH:  ../data/seed_2/p_4/
INFO FILE PATH:  ../data/seed_2/p_4/info.csv
RATINGS FILE PATH:  ../data/seed_2/p_4/ratings.csv
AUTHORS FILE PATH:  ../data/seed_2/p_4/authors.csv


In [48]:

# Papers distribution generation

print("---------- PAPER DISTRIBUTIONS GENERATION STARTED ----------")

paper_distributions = np.empty(papers_number)
for index in range(0, papers_number):
    percentage = 100*index/papers_number
    if percentage % 10 == 0:
        print(f"{int(index)}/{papers_number} ({int(percentage)}/100%)")
    distribution = tn(0, 1, loc=rn.uniform(0, 1), scale=rn.uniform(0, 0.05)).rvs(1)
    paper_distributions[index] = distribution
print(f"{papers_number}/{papers_number} (100/100%)")
    
print("---------- PAPER DISTRIBUTIONS GENERATION COMPLETED ----------")


---------- PAPER DISTRIBUTIONS GENERATION STARTED ----------
0/20000 (0/100%)
2000/20000 (10/100%)
4000/20000 (20/100%)
6000/20000 (30/100%)
8000/20000 (40/100%)
10000/20000 (50/100%)
12000/20000 (60/100%)
14000/20000 (70/100%)
16000/20000 (80/100%)
18000/20000 (90/100%)
20000/20000 (100/100%)
---------- PAPER DISTRIBUTIONS GENERATION COMPLETED ----------


In [49]:

# Ratings file generation

# N sets of readers, each one has X% of the total

readers_percent = 20
reader_sets_number = m.floor(100 / readers_percent)
readers_amount = m.floor((readers_number*readers_percent)/100)

readers_set = set(readers)
readers_sets = []

# Readers rate papers with a certain frequence

paper_frequencies = [2, 4, 8, 30, 90]

print("---------- READERS SETS GENERATION STARTED ----------")

ratings_number = sum(paper_frequencies) * readers_amount
for x in range(0, reader_sets_number):
    current_readers_set = rn.sample(readers_set, readers_amount)
    # Removing last index
    if readers_number in current_readers_set: current_readers_set.remove(readers_number)
    readers_sets.append(current_readers_set)
    for reader in current_readers_set:
        readers_set.remove(reader)
    print(f"SET {x}: ", current_readers_set)
     
print("---------- READERS SETS GENERATION COMPLETED ----------")

print("---------- RATINGS GENERATION STARTED ----------")

generated_ratings = 0
with open(ratings_file_path, mode='w', newline='') as ratings_file:
    ratings_writer = csv.writer(ratings_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    ratings_writer.writerow(['Timestamp', 'Reader', 'Paper', 'Score'])
    for current_set in range(0, reader_sets_number):
        paper_per_reader = paper_frequencies[current_set]
        readers_set = readers_sets[current_set]
        for reader in readers_set:
            for index, paper in enumerate(rn.sample(set(papers), paper_per_reader)):
                paper_distribution = paper_distributions[paper]
                percentage = 100*generated_ratings/ratings_number
                if percentage % 10 == 0:
                    print(f"{int(generated_ratings)}/{ratings_number} ({int(percentage)}/100%)")
                current_tuple = {
                    "Reader": reader, 
                    "Paper": paper, 
                    "Score": round(paper_distribution, 2), 
                }
                ratings_writer.writerow([generated_ratings, current_tuple["Reader"], current_tuple["Paper"], current_tuple["Score"]])
                generated_ratings+=1
    print(f"{ratings_number}/{ratings_number} (100/100%)")
ratings_file.close()

print("---------- RATINGS GENERATION ENDED ----------")


---------- READERS SETS GENERATION STARTED ----------
SET 0:  [454, 2188, 978, 2395, 1258, 418, 1032, 1227, 833, 805, 2358, 2362, 2330, 741, 603, 727, 1776, 1078, 2344, 929, 1585, 1302, 2143, 2292, 1281, 885, 950, 1009, 279, 2278, 1845, 627, 1994, 72, 1190, 890, 2397, 1843, 1954, 2464, 323, 498, 2394, 2036, 1829, 1557, 1433, 2059, 768, 1552, 651, 908, 2401, 2199, 2260, 1807, 694, 1773, 1460, 1312, 240, 968, 553, 2450, 2107, 1077, 586, 905, 1439, 1103, 1798, 1197, 426, 1288, 495, 685, 2297, 2389, 1772, 2432, 1765, 2475, 321, 307, 2120, 816, 1617, 2077, 218, 667, 1381, 1147, 1556, 88, 221, 227, 216, 1220, 129, 1591, 502, 1459, 1503, 1412, 754, 977, 1551, 2159, 1723, 2032, 1587, 428, 941, 1507, 17, 340, 2371, 1978, 1021, 1289, 2021, 115, 114, 186, 1464, 1506, 1267, 1825, 1790, 1099, 1886, 1519, 1918, 864, 2276, 2147, 2400, 343, 1818, 631, 745, 1527, 1801, 1325, 1933, 2426, 660, 1898, 2273, 761, 1421, 13, 1062, 509, 1256, 308, 1327, 550, 2153, 740, 2171, 1080, 2255, 2247, 2215, 1759, 789, 

In [50]:

# Authors file generation

print("---------- AUTHORS GENERATION STARTED ----------")

with open(authors_file_path, mode='w', newline='') as authors_file:
    authors_writer = csv.writer(authors_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    authors_writer.writerow(["Author", "Paper"])
    for index, author in enumerate(authors):
        percentage = 100*index/authors_number
        if percentage % 10 == 0:
            print(f"{int(index)}/{authors_number} ({int(percentage)}/100%)")
        # An author writes a number of paper between 1 and paper_fraction
        author_papers_number = rn.randint(1, (papers_number-1))
        papers_written = np.random.choice(papers, author_papers_number).tolist()
        papers_written = set(papers_written)
        if len(papers_written) > 1:
            papers_written = map(str, list(papers_written))
            papers_written = ";".join(papers_written)
        authors_writer.writerow([author, papers_written])
    print(f"{authors_number}/{authors_number} (100/100%)")
authors_file.close()
        
print("---------- AUTHORS GENERATION ENDED ----------")


---------- AUTHORS GENERATION STARTED ----------
0/25 (0/100%)
5/25 (20/100%)
10/25 (40/100%)
15/25 (60/100%)
20/25 (80/100%)
25/25 (100/100%)
---------- AUTHORS GENERATION ENDED ----------


In [51]:

# Info file generation

print("---------- INFO GENERATION STARTED ----------")

info_dataframe = pd.DataFrame(columns=["Dataset", "Paper", "Reader", "Rating", "Author"])
info_dataframe = info_dataframe.append(
    {
        "Dataset": dataset_name, 
        "Paper": papers_number, 
        "Reader": readers_number, 
        "Rating": ratings_number, 
        "Author": authors_number
    }, ignore_index=True)
info_dataframe.to_csv(info_file_path, index=False)

print("---------- INFO GENERATION ENDED ----------")

---------- INFO GENERATION STARTED ----------
---------- INFO GENERATION ENDED ----------
